# Spotify BPM-Based Playlist Generator

## What does this code do?
I like to run while listening to a BPM playlist to keep a consistent cadence. This code creates a BPM based 
playlist that only uses my liked/saved songs in Spotify so I don't have to manually search for good running songs.

## Setup
- Register a Spotify dev account: https://developer.spotify.com/dashboard
- Create a new app and get the client ID, client secret, redirect URI (I used `http://localhost:9000` for that)
- Set the following environment variables on your local machine accordingly: `SPOTIPY_CLIENT_ID`, `SPOTIPY_CLIENT_SECRET`, `SPOTIPY_REDIRECT_URI` 

## Possible extensions:
- Use the 'Danceability' property in audio_features to get more danceable tracks
- Filter for a specific genre 
- Also allow half-tempo tracks (e.g. 85bpm when you're putting together a 170bpm playlist) since that still works for running.


In [49]:
import random
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Set up the Spotify API client
scope = "playlist-modify-public user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

# Create a new playlist
playlist_name = "My 170 BPM Playlist"
playlist_description = "A playlist of 100 tracks with a BPM of 170"
user_id = sp.current_user()["id"]
playlist = sp.user_playlist_create(user_id, playlist_name, public=True, description=playlist_description)

# Get all tracks in the user's library
results = sp.current_user_saved_tracks()
liked_tracks = results['items']

while results['next']:
    results = sp.next(results)
    liked_tracks.extend(results['items'])
    
print(f'Found {len(liked_tracks)} liked tracks.\n')
  
bpm = 170
bpm_tracks = []

for track in liked_tracks:
    try:
        track_info = track['track']
        track_id = track_info['id']
        track_name = track_info['name']
        track_artist = track_info['artists'][0]['name']
        audio_features = sp.audio_features(track_id)[0]
        if audio_features:
            tempo = audio_features['tempo']

        if abs(tempo - bpm) < 3: # Allow for a small margin of error
            print(f"{track_id}: {track_name} - {track_artist}, BPM: {tempo}")
            bpm_tracks.append(track_info['id'])
    except:
        pass
        

# # Add 100 random tracks with a BPM of 170 to the playlist
print(f'\nFound {len(bpm_tracks)} bpm tracks.')
random_tracks = random.sample(bpm_tracks, min(len(bpm_tracks), 100))
sp.playlist_add_items(playlist['id'], random_tracks)

print(f'\nAdded tracks to {playlist_name}.')

Found 5988 liked tracks.

1MV5BzujqyRQ417qj0rXNS: I Will Dare - The Replacements, BPM: 167.709
4k6Uh1HXdhtusDW5y8Gbvy: Bad Habit - Steve Lacy, BPM: 168.946
6e8qhqWzPgdPMFlOuP7nuC: Doot Doot - Freur, BPM: 167.819
7BYqVvoXpQFhs4jJ0qqNZt: Entre dos tierras - Heroes Del Silencio, BPM: 166.095
7aWKZso3TmfIOZOcL0UrAW: Four Dreams - Jesca Hoop, BPM: 165.059
7HaNlbEP3u3WAxNxVC6M5m: Moon And Moon - Bat For Lashes, BPM: 168.745
1YzN8VARoxF7Z0Fsw7HmhC: First Reactions After Falling Through the Ice - La Dispute, BPM: 168.424
2XxsFWFCGA7Wi9Hv3dVcdF: The Child We Lost 1963 - La Dispute, BPM: 167.443
3jxExv27cf3OSt0BXtAR6Z: Chewing Gum - Blood Orange, BPM: 168.265
4Kh8w1MgC7LYHSbSX2lDzP: Make It Forever - George Clanton, BPM: 168.038
2Kxcoer1xviFJEchTk4NX1: Breakfast of Champions - Jean Grae, BPM: 166.354
72BNAxmvQI3vYPgf9hgnYD: Kiss Me on the Bus - 2008 Remaster - The Replacements, BPM: 165.596
73OIUNKRi2y24Cu9cOLrzM: On the Run - Pink Floyd, BPM: 165.393
0FQBHgOPjfF65YiS8uvOTN: Inspiration Informat